# INFO 2950: Data Cleaning
#### Group Members: Anusha Bishayee, Katheryn Ding

---
### __Data Collection and Cleaning:__

In [1]:
import contextlib
import os
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import yfinance as yf
from scipy import stats
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, root_mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

our original dataset with ESG information for different large/mid-cap companies came in a csv format, which we downloaded from Kaggle. this had about 722 rows, each corresponding to a unique publicly traded company. further description of the columns here can be found in the 'Dataset Description' portion of this notebook.

we first dropped all rows that had null values, which eliminated 27 companies. we then filtered this dataset for just USD currency, excluding companies that are traded in CNY or any other currency. this allows us to have greater familiarity with the industries and companies we analyze - this process eliminated 15 more of our rows, and left us with 680 companies.

In [2]:
esg = pd.read_csv("esg_data.csv")
print(f"original data shape: {esg.shape}")

esg = esg.dropna()
print(f"non-null data shape: {esg.shape}")

esg = esg[esg["currency"] == "USD"]
print(f"refined data shape: {esg.shape}")

original data shape: (722, 21)
non-null data shape: (695, 21)
refined data shape: (680, 21)


then, we converted the 'last_processing_date' column in our dataset to DateTime format - a lot of rows had a differing date formats as well, so we had to convert them all to m/d/y. after that, we sorted the dataset by ascending and descending 'last_processing_date' to see the range of processing dates in the data. 

In [3]:
esg["last_processing_date"] = pd.to_datetime(esg["last_processing_date"], format = "mixed")
esg["last_processing_date"] = esg["last_processing_date"].dt.strftime('%m-%d-%Y')

esg = esg.sort_values(by = "last_processing_date", ascending = False)
print(f"latest dates:\n{esg["last_processing_date"].head(2)}")

esg = esg.sort_values(by = "last_processing_date", ascending = True)
print(f"\nearliest dates:\n{esg["last_processing_date"].head(2)}")

latest dates:
720    11-15-2022
716    11-15-2022
Name: last_processing_date, dtype: object

earliest dates:
658    02-08-2022
36     04-16-2022
Name: last_processing_date, dtype: object


after this, we realized we have two 'Energy' values for the 'industry' column - one is 'Energy ' and one is 'Energy'. we renamed all the 'Energy ' values, and also re-formatted some other industry column values.

In [4]:
esg['industry'] = esg['industry'].replace('Energy ', 'Energy')
esg['industry'] = esg['industry'].replace('Hotels, Restaurants & Leisure', 'Hotels, Restaurants, & Leisure')
esg['industry'] = esg['industry'].replace('Hotels Restaurants and Leisure', 'Hotels, Restaurants, & Leisure')
esg['industry'] = esg['industry'].replace('Consumer products', 'Consumer Products')
esg['industry'] = esg['industry'].replace('Logistics and Transportation', 'Logistics & Transportation')
esg['industry'] = esg['industry'].replace('Life Sciences Tools and Services', 'Life Sciences Tools & Services')
esg['industry'] = esg['industry'].replace('Commercial Services and Supplies', 'Commercial Services & Supplies')
esg['industry'] = esg['industry'].replace('Road and Rail', 'Road & Rail')
esg['industry'] = esg['industry'].replace('Metals and Mining', 'Metals & Mining')
esg['industry'] = esg['industry'].replace('Aerospace and Defense', 'Aerospace & Defense')
esg['industry'] = esg['industry'].replace('Textiles Apparel and Luxury Goods', 'Textiles, Apparel, & Luxury Goods')
esg['industry'] = esg['industry'].replace('Trading Companies and Distributors', 'Trading Companies & Distributors')

print(esg['industry'].unique())

['Leisure Products' 'Semiconductors' 'Health Care' 'Chemicals'
 'Telecommunication' 'Consumer Products' 'Airlines' 'Insurance'
 'Communications' 'Building' 'Technology' 'Logistics & Transportation'
 'Biotechnology' 'Banking' 'Pharmaceuticals' 'Financial Services'
 'Life Sciences Tools & Services' 'Electrical Equipment' 'Real Estate'
 'Machinery' 'Retail' 'Food Products' 'Industrial Conglomerates'
 'Hotels, Restaurants, & Leisure' 'Utilities' 'Beverages' 'Tobacco'
 'Media' 'Auto Components' 'Energy' 'Commercial Services & Supplies'
 'Packaging' 'Road & Rail' 'Metals & Mining'
 'Textiles, Apparel, & Luxury Goods' 'Trading Companies & Distributors'
 'Aerospace & Defense' 'Automobiles' 'Distributors'
 'Professional Services' 'Construction' 'Marine'
 'Diversified Consumer Services']


now, we wanted to add our finance data from the yfinance library onto to our esg dataset. we used the ticker column to match up companies from the yfinance library and our esg dataset, and we set our dates of the finance data to range from 2/1/21 to 12/31/22, as all of the 'last processing date' values for the esg data range from 2/8/22 to 11/15/22. in specific, we calculated a stock percentage change over this period for each company, a volatility index, a 50-day moving average, and a cumulative return metric.

In [5]:
# prevents some annoying yfinance outputs from printing, gpt was used to assist here
@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

tickers = esg["ticker"].tolist()
stock_data = []

for ticker in tickers:
    try:
        # suppresses all of the outputs when grabbing data from yfinance
        with suppress_output():  
            stock = yf.download(ticker, start = "2021-02-01", end = "2022-12-31", progress = False)
        
        if not stock.empty:
            # get closing price for 01/01/2021 and 12/31/2022
            close_2021_02_01 = stock.loc["2021-02-01"]["Close"] if "2021-02-01" in stock.index else None
            close_2022_12_31 = stock.loc["2022-12-30"]["Close"] if "2022-12-30" in stock.index else None

            # calculating percentage change
            percentage_change = ((stock["Close"].iloc[-1] - stock["Close"].iloc[0]) / stock["Close"].iloc[0]) * 100
            
            # calculating volatility (sd of daily returns)
            daily_returns = stock["Close"].pct_change()
            volatility = daily_returns.std()
            
            # calculating 50-day moving average
            stock["50_day_SMA"] = stock["Close"].rolling(window=50).mean()
            sma_50_day = stock["50_day_SMA"].iloc[-1]
            
            # calculating cumulative return
            cumulative_return = (stock["Close"].iloc[-1] / stock["Close"].iloc[0]) - 1
            
            stock_data.append({
                'ticker': ticker, 
                'start_close': close_2021_02_01,
                'end_close': close_2022_12_31,
                'percentage_change': percentage_change,
                'volatility': volatility,
                '50_day_SMA': sma_50_day,
                'cumulative_return': cumulative_return
            })
            
    # also helps to suppress annoying outputs
    except (yf.YFTzMissingError, yf.YFPricesMissingError):
        pass 

now, we needed to convert the stock data we extracted from yfinance to a dataframe, so that we can merge it with our original esg dataframe.

In [6]:
stock_df = pd.DataFrame(stock_data)
merged_df = esg.merge(stock_df, on = 'ticker', how = 'left')
print(f"current data shape: {merged_df.shape}")
print(f"\n{merged_df.head()}")

current data shape: (680, 27)

  ticker                           name currency  \
0   poww                       Ammo Inc      USD   
1   acls       Axcelis Technologies Inc      USD   
2   achc  Acadia Healthcare Company Inc      USD   
3     cf     CF Industries Holdings Inc      USD   
4      t                       AT&T Inc      USD   

                        exchange           industry  \
0     NASDAQ NMS - GLOBAL MARKET   Leisure Products   
1     NASDAQ NMS - GLOBAL MARKET     Semiconductors   
2     NASDAQ NMS - GLOBAL MARKET        Health Care   
3  NEW YORK STOCK EXCHANGE, INC.          Chemicals   
4  NEW YORK STOCK EXCHANGE, INC.  Telecommunication   

                                                logo  \
0  https://static.finnhub.io/logo/8decc6ca0564a89...   
1  https://static.finnhub.io/logo/88b5f730-80df-1...   
2  https://static.finnhub.io/logo/4b6b2e5a4cfce5b...   
3  https://static.finnhub.io/logo/9b57a636-80eb-1...   
4  https://static.finnhub.io/logo/7d20269e-80

after that, we needed to drop any rows where the finance data join has left null values. this eliminates 80 more of our rows, leaving us with 600 companies.

In [7]:
merged_df = merged_df.dropna()
print(f"non-null finance data shape: {merged_df.shape}\n")

merged_df = merged_df.sort_values(by = "name", ascending = True)
print(merged_df.head)

non-null finance data shape: (597, 27)

<bound method NDFrame.head of     ticker                             name currency  \
71     mmm                            3M Co      USD   
142    aos                   A O Smith Corp      USD   
661   abvc               ABVC Biopharma Inc      USD   
17    acad       ACADIA Pharmaceuticals Inc      USD   
41    aciw                ACI Worldwide Inc      USD   
..     ...                              ...      ...   
500    zts                       Zoetis Inc      USD   
571    zuo                        Zuora Inc      USD   
643    zws  Zurn Elkay Water Solutions Corp      USD   
647   zyme                    Zymeworks Inc      USD   
189   ebay                         eBay Inc      USD   

                          exchange                  industry  \
71   NEW YORK STOCK EXCHANGE, INC.  Industrial Conglomerates   
142  NEW YORK STOCK EXCHANGE, INC.                  Building   
661     NASDAQ NMS - GLOBAL MARKET             Biotechnology   
1

In [8]:
merged_df = merged_df.drop(['exchange', 'logo', 'weburl','cik'],axis=1)
merged_df.columns

Index(['ticker', 'name', 'currency', 'industry', 'environment_grade',
       'environment_level', 'social_grade', 'social_level', 'governance_grade',
       'governance_level', 'environment_score', 'social_score',
       'governance_score', 'total_score', 'last_processing_date',
       'total_grade', 'total_level', 'start_close', 'end_close',
       'percentage_change', 'volatility', '50_day_SMA', 'cumulative_return'],
      dtype='object')

this is still a lot of data, but will be helpful for getting industry-level and other general overviews of the data. merged_df will be our main dataframe.
   
we also want to create a sample of these 600 companies so that we are able to look at trends and associations at the individual company-level as well.   
   
since we want to take random sample stratum on industries and total ESG score:
1. We first list all industries with descending total_score, then separate the industries based on total_score into three groups: high/medium/low ESG based on their rank.
2. Then, we use  .sample() to randomly choose two industries from each group, which generates 6 industries.
3. For each of the randomly chosen industries, we randomly pick 5 companies from each industry, which generates 30 companies in the sample_companies.
   
By applying this sampling process, we ensure our sample by industries is representative of all levels of ESG scores. gpt was used here for assistance with the sampling. sample_companies will be our 2nd dataframe.

In [9]:
warnings.filterwarnings("ignore", category = DeprecationWarning)
np.random.seed(123)

avg_esg_by_industry = merged_df.groupby('industry')['total_score'].mean().reset_index()
avg_esg_by_industry.columns = ['Industry', 'Average Total ESG Score']
avg_esg_by_industry = avg_esg_by_industry.sort_values(by = 'Average Total ESG Score', ascending = False)

third = len(avg_esg_by_industry) // 3
high_group = avg_esg_by_industry.iloc[:third]
medium_group = avg_esg_by_industry.iloc[third: 2 * third]
low_group = avg_esg_by_industry.iloc[2 * third:]

random_high_industries = high_group.sample(2)['Industry'].tolist()
random_medium_industries = medium_group.sample(2)['Industry'].tolist()
random_low_industries = low_group.sample(2)['Industry'].tolist()

print(f"randomly selected high ESG industries: {random_high_industries}")
print(f"randomly selected medium ESG industries: {random_medium_industries}")
print(f"randomly selected low ESG industries: {random_low_industries}")

high_industry_companies = merged_df[merged_df['industry'].isin(random_high_industries)]
medium_industry_companies = merged_df[merged_df['industry'].isin(random_medium_industries)]
low_industry_companies = merged_df[merged_df['industry'].isin(random_low_industries)]

high_industry_companies_sampled = high_industry_companies.groupby('industry', group_keys = False).apply(lambda x: x.sample(min(len(x), 5))).reset_index(drop = True)
high_industry_companies_sampled['ESG score level'] = 'High'
medium_industry_companies_sampled = medium_industry_companies.groupby('industry', group_keys = False).apply(lambda x: x.sample(min(len(x), 5))).reset_index(drop = True)
medium_industry_companies_sampled['ESG score level'] = 'Medium'
low_industry_companies_sampled = low_industry_companies.groupby('industry', group_keys = False).apply(lambda x: x.sample(min(len(x), 5))).reset_index(drop = True)
low_industry_companies_sampled['ESG score level'] = 'Low'

sample_companies = pd.concat([high_industry_companies_sampled, medium_industry_companies_sampled, low_industry_companies_sampled])
print(sample_companies)

randomly selected high ESG industries: ['Building', 'Road & Rail']
randomly selected medium ESG industries: ['Professional Services', 'Real Estate']
randomly selected low ESG industries: ['Biotechnology', 'Banking']
  ticker                              name currency               industry  \
0     tt            Trane Technologies PLC      USD               Building   
1    mas                        Masco Corp      USD               Building   
2    zws   Zurn Elkay Water Solutions Corp      USD               Building   
3    aos                    A O Smith Corp      USD               Building   
4   aaon                          Aaon Inc      USD               Building   
5    xpo                 XPO Logistics Inc      USD            Road & Rail   
6    unp                Union Pacific Corp      USD            Road & Rail   
7    csx                          CSX Corp      USD            Road & Rail   
8    nsc             Norfolk Southern Corp      USD            Road & Rail   
9   

finally, we also extracted some general S&P 500 data from yfinance, ranging from the dates of 2/1/21 and 12/31/22 for the same reason. we are pulling this data so that we can compare stock performance of the individual companies to the overall S&P 500 in the same time range. sp500 will be our 3rd dataset.

In [10]:
sp500data = yf.download('^GSPC', start = '2021-02-01', end = '2022-12-31', progress = False)
sp500 = pd.DataFrame({
    'Date': sp500data.index,
    'Start Price': sp500data['Open'],
    'End Price': sp500data['Close'],
    'Rate of Change': ((sp500data['Close'] - sp500data['Open']) / sp500data['Open']) * 100 })

sp500.set_index('Date', inplace = True)
print(sp500)

ValueError: Data must be 1-dimensional, got ndarray of shape (484, 1) instead

now, we will convert some of our dataframes into csv to use in the next stages of our project.

In [ ]:
sp500.to_csv('sp500.csv', index = False) 
sample_companies.to_csv('sample_companies.csv', index = False) 
avg_esg_by_industry.to_csv('avg_esg_by_industry.csv', index = False) 
merged_df.to_csv('merged_df.csv', index = False)